In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
import statsmodels as sm
from tqdm import tqdm
import urllib
import urllib2, requests
import pandas as pd
import re

#--------------------------------------------------------------------------------------------------------------    
#Scraping lat & lng from each apartment:

#Open the refer csv file and create a list with the reference number:
df = pd.read_csv("../datasets/refer")
lista = list(df["0"])
lista1 = lista[0:89601]
#Create a list with the urls that we will extract the data:

url = []
for i in lista1:
    url.append("http://www.squarefoot.com.hk/property/" + str(i))
    
#Dictionary with the url and the ref number:
lel = dict(zip(url, lista1))

#Iterate in the dictionary and save the html files in our neverend folder:
for i, j in tqdm(lel.iteritems()):
    try:
        html_content = urllib2.urlopen(i)
        content = html_content.read()
        folder = "neverend"
        filename = folder + '/' + str(j) + ".html" 
        with file(filename, "wt") as fo:
            fo.write(content)
    except Exception:
        pass

#Checking that there are no offers repeated for the same apartment:
ue = list(df["0"].unique())

#List with the address of our html files:
refdoc = []
for i in ue:
    x = "file:///C:/Users/alexrebull/.babun/cygwin/home/alexrebull/DSI-HK-1/capstone/Assets/scraping/neverend/" + str(i) + ".html"
    refdoc.append(x)
    
#Extract the latitude and longitude for each one of our html files:
lat = []
lng = []
for i in tqdm(refdoc):
    try:
        word = "LatLng((.*))"
        contents = BeautifulSoup(urllib.urlopen(i), "lxml")  
        contentsElements = contents.find_all('script', attrs={'type' : 'text/javascript'})
        for t in contentsElements:
            for u in t:
                for p in t:
                    if "LatLng" in p:
                        print p
                        le = "".join(p)
                        pos = re.search(word, le)
                        coord = pos.group(1)
                        lat.append(coord.split(",")[0])
                        lng.append(coord.split(",")[1])
                        
    except:
        lat.append(None)
        lng.append(None) 
        
#Cleaning the lat and lng features:
lat1 = []
for i in lat:
    if i != None:
        lat1.append(i.split("(")[1])
    else:
        lat1.append(None)
        
lng1 = []
for i in lng:
    if i != None:
        lng1.append(i.split(")")[0])
    else:
        lng1.append(None)
        
#Create dictionaries for lat and lng, merge both dataframes and save it as a csv file called "coordapart":
dictlat = dict(zip(ue, lat1))
dictlng = dict(zip(ue, lng1))

dflat = pd.DataFrame(dictlat.items(), columns=['ref', 'lat'])
dlng = pd.DataFrame(dictlng.items(), columns=["ref", "lng"])

result = pd.merge(dflat, dlng, how='inner', on=["ref"])

result.to_csv("datasets/coordapart")

#--------------------------------------------------------------------------------------------------------------    
#Scraping the data from each apartment:
layout = []
floor = []
area = []
age = []
address = []
manfee = []
rate = []
out_area = []
bath = []
attr = []
ref = []
for i in tqdm(refdoc):
    form = []
    try:
        contents = BeautifulSoup(urllib2.urlopen(i), "lxml")
        contentsElements = contents.findAll('table', attrs={'id':'DetailTable'})
        for i in contentsElements:
            xp = i.findAll("td", attrs={"class" : "output"})
            for t in xp:
                form.append(t.getText())
            if len(form) == 15:
                layout.append(form[1])
                floor.append(form[2])
                area.append(form[4])
                address.append(form[6])
                age.append(form[7])
                manfee.append(form[10])
                rate.append(form[11])
                out_area.append(form[12])
                bath.append(form[14])
            else:
                layout.append(form[1])
                floor.append(form[2])
                area.append(form[4])
                address.append(form[8])
                age.append(form[9])
                manfee.append(form[12])
                rate.append(form[13])
                out_area.append(form[14])
                bath.append(form[16])
                
        contentsElements1 = contents.findAll('table', attrs={'border':'0'})
        for p in contentsElements1:
            attr.append("".join(p.getText()))
        contentsElements2 = contents.findAll('div', attrs={'class':'propertyTitle'})
        for pl in contentsElements2:
            ref.append(pl.getText().split()[-1])
    except:
        next
       
    
#Cleanining some features:
floor1 = []
area1 = []
address1 = []
age1 = []
bath1= []

for i in floor:
    floor1.append(i.split("\r\n")[1].strip())
for i in area:
    area1.append(i.split("\r\n")[1].strip()) 
for i in address:
    address1.append(i.split("\r\n")[1].strip())  
for i in age:
    age1.append(i.split("\r\n")[1].strip())
for i in bath:
    bath1.append(i.split("\r\n")[1].strip())

#Create a dataframe and save it as a csv file:
df = pd.DataFrame({"ref" : ref, "floor": floor1, "area" : area, "age" : age1, "bath" : bath1, "attr" = attr})

df.to_csv("datasets/smalldf")